In [37]:
import os
import pickle
import numpy as np
from itertools import permutations
import pandas as pd
import pingouin as pg

# statistical methods
from scipy.stats import chisquare
from scipy.stats import ttest_1samp
from scipy.stats import ttest_ind
from scipy.stats import f_oneway

from bioinfokit.analys import stat

import statsmodels.stats as st

In [5]:
# read all the backups
address = "../data/equicity_kabeldistrict/backup"
game_backups = os.listdir(address)
#game_backups.pop(game_backups.index('.DS_Store'))
# sort the folders in ascending order
game_backups.sort()

In [6]:
game_id = -1
# load project and users dict
project_dict = pickle.load( open( address + "/" + game_backups[game_id] + "/project.pickle", "rb" ) )
users_dict = pickle.load( open( address + "/" + game_backups[game_id] + "/users.pickle", "rb" ) )

In [7]:
def print_keys(d, l=0):
    # iterate over the dict keys
    for k in d.keys():
        # extract the shape if the value is a list
        shape = np.array(d[k]).shape if isinstance(d[k],list) else ""
        # print info
        print(l*"  ", k, ":", type(d[k]), shape)
        # recurse if the value is a dict
        if isinstance(d[k],dict):
            print_keys(d[k], l+1)

In [8]:
print_keys(project_dict)

 analysis : <class 'dict'> 
   CC : <class 'dict'> 
     1638367010519 : <class 'list'> (7, 5)
     1638368175838 : <class 'list'> (7, 5)
     1638370659848 : <class 'list'> (7, 5)
   CCt : <class 'list'> (7, 5)
 current_host : <class 'str'> 
 game_info : <class 'dict'> 
   contributor : <class 'str'> 
   gainer : <class 'str'> 
   message : <class 'str'> 
   player : <class 'str'> 
   round : <class 'int'> 
   scores : <class 'dict'> 
     change_score : <class 'dict'> 
       0 : <class 'int'> 
       1638367010519 : <class 'float'> 
       1638368175838 : <class 'float'> 
       1638370659848 : <class 'float'> 
     change_score_t : <class 'float'> 
     closeness_score : <class 'dict'> 
       0 : <class 'int'> 
       1638367010519 : <class 'float'> 
       1638368175838 : <class 'float'> 
       1638370659848 : <class 'float'> 
     closeness_score_t : <class 'float'> 
     environmental_score : <class 'dict'> 
       0 : <class 'int'> 
       1638367013122 : <class 'float'> 
   

## Analysis Outline

### Questions!

1. are the site and actor homogenous:
    * 1.1. how homogenous are the decisions of the participants (considering that the roles are different)?
    * 1.2. how homogeneously the sites are being treated by the decisions of the participants (considering that they have different potentials)? (this will be assessed by the type of allocation/investment that is being proposed for each of them)
2. how different are the decisions of the participants from round to round?
3. are the players improving their scores through the game play (generally)? (for this question also we can combine the info from all the games)
4. will longer discussion times result in larger change in their decision or their score?
    * for this question the result of all workshops can be combined together

### Considerations

* 1st question: the two sub questions can be combined as a two level [ANOVA](https://en.wikipedia.org/wiki/Analysis_of_variance), and the participants and sites will be the main effect and their intersection will be the interaction/cross-effect
* 2nd question: since the main format of the data is not vectors but matrices, we need to unroll the matrices into one dimensional arrays, for that we need [Quadratic Assignment Process](https://ideas.repec.org/p/boc/asug01/1.2.html). So first we need to permute (changing the order of rows and columns) randomly for 1K variations, and then we need to unroll these matrices in the same way, and run the statistical analysis, which will result in a distribution of statistics, and then the p value can be estimated. 
* in statistics we have to be wary of false positive and false negatives, by repeating the test several times, we would increase the chance of falling into false positive, this is more relevant to change of scores (question 3, 4), yet since we are performing this for multiple tests already, it means that we need to adjust the p-value, which is called [bonferroni correction](https://en.wikipedia.org/wiki/Bonferroni_correction) or another correction in the same family
* for the chi-square, if we are performing multiple times (multiple means for different rounds, different participants, and sites), then the collective test needs to be executed first ([ANOVA](https://en.wikipedia.org/wiki/Analysis_of_variance) family such as H test), then only if this significant then we perform the post-hoc individual test
* regarding the methods, we are only 100% sure about the chi-square and the pearsman, the rest need to be checked

### Methods

* 1 and 2:
    * (individual, based on each round) since it is categorical data, [chi-square](https://en.wikipedia.org/wiki/Chi-squared_test), with the assumption that the next round is expected, and this round is the observed set, so we can measure the dif (this is probably the most relevant)
    * [Mann–Whitney U test](https://en.wikipedia.org/wiki/Mann–Whitney_U_test) is similar to chi-square (chi-square is potentially better since U test is related to ordinal data)
    * (overall) [Kruskal-Wallis H tests](https://en.wikipedia.org/wiki/Kruskal–Wallis_one-way_analysis_of_variance), a kind of an analysis of variance, it is to compare different groups of U test (this is also can be checked to see if it can extend chi-square (it normally extents the U test))

* 3:
    * measure the positive change of every round for each score separately, and perform a [t test](https://en.wikipedia.org/wiki/Student%27s_t-test) and compare the distribution to zero, since we want to know if it is significantly changing, 
    * the hypothesis is that whether the change is larger than zero
    
* 4:
    * for assessing the change in the scores since they are numerical, it is just a correlation: [pearson](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient), (or [spearman](https://statistics.laerd.com/statistical-guides/spearmans-rank-order-correlation-statistical-guide.php))
    * for the changes in the decision, since they are categorical, an approach to assess the change in the categorical data, we can use [earth movers distance](https://en.wikipedia.org/wiki/Earth_mover%27s_distance) (check if this can be used to assess the first question)

### Sources to study statistics
* statistical methods for geography, peter rogerson
* spatial statistics for geography, peter rogerson

* they use SPSS, but scipy is sufficient to replace that

## Analysis Implementation

### Q1
**are the site and actor homogenous with respect to decisions?**

In [9]:
# extract the decision dictionary
decision_dict = project_dict["gameplay"]["X"]
# remove the 0 time stamp
_ = decision_dict.pop("0", None)
# extract and sort time stamps of decisions
dec_ts_sorted = np.array(list(decision_dict.keys()), dtype=int)
dec_ts_sorted.sort()
# extract a sample decision
x0 = np.array(decision_dict[str(dec_ts_sorted[0])])
# construct a 4 dimensional tensor containting all the observations:
# (Rounds, Actors, Sites, Colors)
X = np.array([decision_dict[str(ts)] for ts in dec_ts_sorted])

#### Q1.1
**how homogenous are the decisions of the participants (considering that the roles are different)?**

In [7]:
# transpose the X (decision tensor) to the following shape:
# (Actor, Site, Color, Round)
X_A = X.transpose(1,2,3,0)
# flatten all the dimensions except the actor dimension
X_a = X_A.reshape((X_A.shape[0], -1))

In [8]:
# H0: there is no significant difference between the decisions of different players
# H1: there is a significant difference between the decision of the players
# alpha: 0.05
ALPHA = 5e-2
# number of decision entries (decision variables)
n = X_a.shape[1]
# number of observations in total
N = X_a.size
# degrees of freedom
df_between =  N//n - 1
df_within = N - N//n
df_total = N - 1 # : df_between + df_within
(df_between, df_within)

(4, 520)

In [9]:
r = f_oneway(*tuple(X_a))
print("statistic :", r.statistic)
print("p-value :", r.pvalue)
print("H0 is", ALPHA < r.pvalue, "and H1 is", ALPHA > r.pvalue)

statistic : 6.851694034651903e-32
p-value : 1.0
H0 is True and H1 is False


#### Q1.2
**how homogeneously the sites are being treated by the decisions of the participants (considering that they have different potentials)?**

In [10]:
# transpose the X (decision tensor) to the following shape:
# (Site, Actor, Color, Round)
X_S = X.transpose(2,1,3,0)
# flatten all the dimensions except the actor dimension
X_s = X_S.reshape((X_S.shape[0], -1))

In [11]:
# H0: there is no significant difference between the decisions about different sites
# H1: there is a significant difference between the decision about sites
# alpha: 0.05
ALPHA = 5e-2
# number of decision entries (decision variables)
n = X_s.shape[1]
# number of observations in total
N = X_s.size
# degrees of freedom
df_between =  N//n - 1
df_within = N - N//n
df_total = N - 1 # : df_between + df_within
(df_between, df_within)

(6, 518)

In [12]:
r = f_oneway(*tuple(X_s))
print("statistic :", r.statistic)
print("p-value :", r.pvalue)
print("H0 is", ALPHA < r.pvalue, "and H1 is", ALPHA > r.pvalue)

statistic : -1.249130264267867e-31
p-value : nan
H0 is False and H1 is False


#### Q1 
(sub-questions combined using two-way ANOVA)

In [10]:
# convert the X to pandas dataframe
X_ind = np.indices(X.shape)
X_ind_flat = X_ind.reshape(X_ind.shape[0], -1)
X_df = pd.DataFrame(X_ind_flat.T, columns=["Round", "Actor", "Site", "Color"])
X_df["x"] = X.ravel()
X_df

,Round,Actor,Site,Color,x
0,0,0,0,0,0.152381
1,0,0,0,1,0.390476
2,0,0,0,2,0.200000
3,0,0,0,3,0.200000
4,0,0,0,4,0.057143
...,...,...,...,...,...
520,2,4,6,0,0.152381
521,2,4,6,1,0.104762
522,2,4,6,2,0.390476
523,2,4,6,3,0.247619


##### Descriptive Statistics of the three levels for Actor, Site, and Color

In [47]:
X_df[['Actor','x']].groupby('Actor').mean(), X_df[['Actor','x']].groupby('Actor').std()

(         x
 Actor     
 0      0.2
 1      0.2
 2      0.2
 3      0.2
 4      0.2,
               x
 Actor          
 0      0.062298
 1      0.150150
 2      0.140083
 3      0.178907
 4      0.070507)

In [48]:
X_df[['Site','x']].groupby('Site').mean(), X_df[['Site','x']].groupby('Site').std()

(        x
 Site     
 0     0.2
 1     0.2
 2     0.2
 3     0.2
 4     0.2
 5     0.2
 6     0.2,
              x
 Site          
 0     0.140260
 1     0.116642
 2     0.117689
 3     0.067344
 4     0.089259
 5     0.116642
 6     0.208451)

In [49]:
X_df[['Color','x']].groupby('Color').mean(), X_df[['Color','x']].groupby('Color').std()

(              x
 Color          
 0      0.141497
 1      0.193651
 2      0.227664
 3      0.239002
 4      0.198186,
               x
 Color          
 0      0.094146
 1      0.120697
 2      0.125082
 3      0.102468
 4      0.166264)

##### Checking for Assumptions of ANOVA

First check for the homogeneity of the data for equal variance

In [58]:
res = stat()
res.levene(df=X_df, res_var='x', xfac_var=['Actor','Site','Color'])
res.levene_summary

,Parameter,Value
0,Test statistics (W),0.7974
1,Degrees of freedom (Df),174.0000
2,p value,0.9541


The dataset with three-way design has equal variance with Levene test (*W*=.797, *df*=174, *p*=0.954). Normal ANOVA is used for later comparison.

##### ANOVA

In [60]:
# three-way ANOVA usign Pingouin library
aov = pg.anova(data=X_df, dv='x', between=['Actor', 'Site', 'Color'], detailed=True,effsize='np2').round(3)
aov

,Source,SS,DF,MS,F,p-unc,np2
0,Actor,0.000,4.0,0.000,0.000,1.0,0.000
1,Site,0.000,6.0,0.000,0.000,1.0,0.000
2,Color,0.604,4.0,0.151,42.276,0.0,0.326
3,Actor * Site,0.000,24.0,0.000,0.000,1.0,0.000
4,Actor * Color,2.834,16.0,0.177,49.596,0.0,0.694
5,Site * Color,1.180,24.0,0.049,13.762,0.0,0.486
6,Actor * Site * Color,2.766,96.0,0.029,8.068,0.0,0.689
7,Residual,1.250,350.0,0.004,NaN,NaN,NaN


The three-way ANOVA shows that the main effects of Actor and Site are not significant: *F*<sub>a</sub>(4,350)=0, *p*>.05,$\eta^2$=0; *F*<sub>s</sub>(6,350)=0, *p*>.05,$\eta^2$=0; the main effect of Color is significant: *F*<sub>c</sub>(4,350)=42.276, *p*<.001,$\eta^2$=.326.

### Q2
**how different are the  decisions of the participants from round to round?**

Since the Q2 is also similar to the questions of Q1, I have integrated them into a three-way ANOVA.

In [15]:
# three-way ANOVA usign Pingouin library
aov = pg.anova(data=X_df, dv='x', between=['Actor', 'Site', 'Round'], detailed=True)
aov

,Source,SS,DF,MS,F,p-unc,np2
0,Actor,1.216071e-29,4.0,3.040179e-30,1.478734e-28,1.0,1.408318e-30
1,Site,1.856933e-30,6.0,3.094889e-31,1.505345e-29,1.0,2.150492e-31
2,Round,7.801732e-31,2.0,3.900866e-31,1.897370e-29,1.0,9.035095e-32
3,Actor * Site,4.890022e-30,24.0,2.037509e-31,9.910385e-30,1.0,5.663077e-31
4,Actor * Round,6.959483e-30,8.0,8.699353e-31,4.231340e-29,1.0,8.059695e-31
5,Site * Round,4.753370e-31,12.0,3.961142e-32,1.926688e-30,1.0,5.504822e-32
6,Actor * Site * Round,1.277656e-29,48.0,2.661783e-31,1.294683e-29,1.0,1.479638e-30
7,Residual,8.634921e+00,420.0,2.055933e-02,NaN,NaN,NaN


### Q3
**are the players improving their scores through the game play (generally)?**

In [16]:
# extract the scores
score_dict = project_dict["game_info"]["scores"]
# remove the recent scores
[score_dict.pop(k, None) for k in ['change_score_t',  'closeness_score_t', 'environmental_score_t', 'individual_score_t']]
# extract individual scores
indiv_score = score_dict.pop("individual_score", None)
# separate valid scores
valid_scores = {}
for k, v in indiv_score.items():
    if isinstance(v, dict):
        valid_scores[k[:5]] = v
# update the score dictionary with the calid scores
score_dict.update(valid_scores)

In [17]:
unified_score_dict = {}
# simplify the timestamps to unify
for k, v in score_dict.items():
    local_score_dict = {}
    for k0, v0 in v.items():
        local_score_dict[str(int(int(k0)*1.e-5))] = v0
    unified_score_dict[k] = local_score_dict

In [18]:
# convert the data dictionary 
scores_df = pd.DataFrame(unified_score_dict).drop("0")
scores_df

,change_score,closeness_score,environmental_score,2rH45,9gDju,Ccmr7,LNYUh,gHqMt
16383670,0.679600,0.314523,0.850648,0.172787,0.284601,0.330133,0.418206,0.347863
16383681,0.678689,0.308280,0.850327,0.171625,0.285477,0.329160,0.417075,0.347863
16383706,0.655034,0.299094,0.824199,0.166619,0.286388,0.325213,0.408819,0.347863


In [19]:
# iterate over the rounds
for i in range(len(scores_df) - 1):
    # extract the scores of the rounds
    s0 = scores_df.iloc[i].to_numpy()
    s1 = scores_df.iloc[i+1].to_numpy()
    # run the t test
    obs = ttest_1samp(s1 - s0 , 0)
    print("round :", i)
    print("statistic :", obs.statistic)
    print("p-value :", obs.pvalue)

round : 0
statistic : -1.6288181349765882
p-value : 0.14737518610312203
round : 1
statistic : -2.609286812560844
p-value : 0.034949586466993886


#### Q4
**will longer discussion times result in larger change in their decision or their score?**

In [20]:
# extract the index as time
scores_df["time"] = scores_df.index
# convert the type from string to int
scores_df = scores_df.astype({'time': 'int'})
# compute the difference between rounds of the game
scores_diff = scores_df.diff()
# drop the NaN row
scores_diff = scores_diff.dropna()

In [21]:
# compute the pearson correlation
scores_diff.corr()

,change_score,closeness_score,environmental_score,2rH45,9gDju,Ccmr7,LNYUh,gHqMt,time
change_score,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
closeness_score,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
environmental_score,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
2rH45,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
9gDju,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0
Ccmr7,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
LNYUh,1.0,1.0,1.0,1.0,-1.0,1.0,1.0,-1.0,-1.0
gHqMt,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0
time,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,-1.0,1.0,1.0
